In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.vehicle'):
    shutil.rmtree('./file/3.vehicle')

os.makedirs('./file/3.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Vehicle Page'] = input_['Vehicle Page'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                df_temp, h3_last, page = pd.DataFrame(), '', 1
                while page <= input_.loc[a, 'Vehicle Page']:
                    b = 0
                    while True:
                        b += 1

                        try:
                            request_url = f'''{input_.loc[a, 'Url']}?page={page}#applications'''
                            
                            resp = requests.get(request_url,
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                                
                            if resp.status_code == 200:
                                soup = BeautifulSoup(resp.text, 'lxml')
                                html = etree.HTML(str(soup))
                                
                                if html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip() == input_.loc[a, 'Part Number']:
                                    break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
            
                    # = = = = = = = = = = = = = = =
            
                    list_h3, list_table = [text.strip() for text in html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]/text()')], html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')
            
                    # = = = = = = = = = = = = = = =

                    if not list_table:
                        raise
                        
                    # = = = = = = = = = = = = = = =

                    if len(list_h3) == len(list_table):
                        for (i, h3), table in zip(enumerate(list_h3), list_table):
                            list_th, list_row = [th.xpath('./text()')[0].strip() for th in table.xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in table.xpath('./tbody/tr')]
                            df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                         'Part Number': input_.loc[a, 'Part Number'],
                                                         'Page': page,
                                                         'Vehicle No': i + 1,
                                                         'Make Model Engine': h3,
                                                         'Row': [j+1 for j in range(len(list_row))]})
                
                            for j, row in enumerate(list_row):
                                for k, th in enumerate(list_th):
                                    df_temp_temp.loc[j, th] = row[k]
                
                            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                    elif len(list_h3) + 1 == len(list_table):
                        list_th, list_row = [th.xpath('./text()')[0].strip() for th in list_table[0].xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in list_table[0].xpath('./tbody/tr')]
                        df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                     'Part Number': input_.loc[a, 'Part Number'],
                                                     'Page': page,
                                                     'Vehicle No': -1,
                                                     'Make Model Engine': h3_last,
                                                     'Row': [j+1 for j in range(len(list_row))]})
                
                        for j, row in enumerate(list_row):
                            for k, th in enumerate(list_th):
                                df_temp_temp.loc[j, th] = row[k]
                                
                        df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                        
                        list_table.pop(0)
                        
                        for (i, h3), table in zip(enumerate(list_h3), list_table):
                            list_th, list_row = [th.xpath('./text()')[0].strip() for th in table.xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in table.xpath('./tbody/tr')]
                            df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                         'Part Number': input_.loc[a, 'Part Number'],
                                                         'Page': page,
                                                         'Vehicle No': i + 1,
                                                         'Make Model Engine': h3,
                                                         'Row': [j+1 for j in range(len(list_row))]})
                
                            for j, row in enumerate(list_row):
                                for k, th in enumerate(list_th):
                                    df_temp_temp.loc[j, th] = row[k]
                
                            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                    else:
                        raise

                    # = = = = = = = = = = = = = = =

                    if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 10 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} - {input_.loc[a, 'No']}.{request_url} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
            
                    h3_last = list_h3[-1]

                    page += 1
                        
                # = = = = = = = = = = = = = = =

                if df_temp.empty:
                    raise

                # = = = = = = = = = = = = = = =

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct['Year'] = output_correct['Year'].astype(int)
        
        output_correct = output_correct.sort_values(by=['No', 'Page', 'Vehicle No', 'Row'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.vehicle/{file}',
              f'''./file/3.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[尝试次数：1] - 50.00% - crawler_1 - 5.https://ecat.spectrapremium.com/en/parts/BMP08A?page=1#applications > Page 1
[剩余数量：5] - [当前时间：10:26:19]

[尝试次数：1] - 50.00% - crawler_1 - 2.https://ecat.spectrapremium.com/en/parts/BMP03A?page=1#applications > Page 1
[剩余数量：4] - [当前时间：10:26:19]

[尝试次数：1] - 50.00% - crawler_1 - 3.https://ecat.spectrapremium.com/en/parts/BMP05A?page=1#applications > Page 1
[剩余数量：3] - [当前时间：10:26:19]

[尝试次数：1] - 50.00% - crawler_1 - 1.https://ecat.spectrapremium.com/en/parts/BMP02A?page=1#applications > Page 1
[剩余数量：2] - [当前时间：10:26:19]

[尝试次数：1] - 50.00% - crawler_1 - 4.https://ecat.spectrapremium.com/en/parts/BMP06A?page=1#applications > Page 1
[剩余数量：1] - [当前时间：10:26:20]

[尝试次数：1] - 50.00% - crawler_1 - 6.https://ecat.spectrapremium.com/en/parts/BMP09A?page=1#applications > Page 1
[剩余数量：1] - [当前时间：10:26:21]

[尝试次数：1] - 50.00% - crawler_1 - 7.https://ecat.spectrapremium.com/en/parts/BMP10A?page=1#applications > Page 1
[剩余数量：1] - [当前时间：10:26:21]

[尝试次数：1] - 50.00% -

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4025.24it/s]

Done ~
